In [1]:
import pandas as pd
from myfun.addcol import addcolumns
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
import joblib
import os

#### 以異地同時法做random forest模型訓練程式碼

In [ ]:
df1 = pd.read_csv(f"./train_combine_groupby/01_traingroupby.csv")
df1 = addcolumns(df1)
def combine_weather(df1:pd.DataFrame):  #定義函數將各裝置每個時間點的資訊與中央氣象局每隔10分鐘的氣象資料做merge
    weather_df = pd.read_csv(f"../資料前處理以及第三方資料/10mins_weather_data.csv")
    weather_df["DateTime"] =  pd.to_datetime(weather_df["DateTime"])
    weather_df["日期"] = pd.to_datetime(weather_df["DateTime"].dt.date)
    weather_df["時間"] = weather_df["DateTime"].dt.hour.astype(str).str.zfill(2)
    weather_df["時段"] = weather_df["DateTime"].dt.minute.astype(str).str.zfill(2)
    
    weather_df.fillna(0,inplace=True)
    df = pd.merge(
        df1,
        weather_df,
        left_on=["日期","時間","時段"],
        right_on=["日期","時間","時段"],
        how="left",
        suffixes=["","_r"]
    )

    return(df)
comdf = combine_weather(df1)

In [ ]:
def selecttime(df:pd.DataFrame):   #定義函數挑出訓練資料的時間(可自行調整)
    df["TimeGroup"] = pd.to_datetime(df["TimeGroup"])
    df = df[(df["TimeGroup"].dt.hour >= 9) & (df["TimeGroup"].dt.hour < 17)| ((df["TimeGroup"].dt.hour == 17) & (df["TimeGroup"].dt.minute == 0))]
    return(df)
selecttime(combine_weather(df1)).head(2)

,Serial,LocationCode,TimeGroup,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),日期,...,SunshineDuration,UVIndex,AirTemperature,DailyExtremeHighAirTemperature,DailyExtremeLowAirTemperature,RelativeHumidity,VisibilityDescription,StationLatitude,StationLongitude,StationAltitude
15,20240101090001,1,2024-01-01 09:00:00,1.91588,1017.906,18.33,92.29,8395.250,17.368,2024-01-01,...,0.0,0.0,18.8,19.3,18.6,64.0,7-10,23.975128,121.61327,16.1
16,20240101091001,1,2024-01-01 09:10:00,0.09000,1017.965,18.02,97.37,7046.501,12.612,2024-01-01,...,0.0,0.0,18.9,19.3,18.6,64.0,21-30,23.975128,121.61327,16.1


In [ ]:
d = {}
for i in range(1,18):
    for j in range(1,18):
        if i==j:
            continue
        l1 = "0" + str(i) if i<10 else str(i)
        l2 = "0" + str(j) if j<10 else str(j)
        df1 = pd.read_csv(f"./train_combine_groupby/{l1}_traingroupby.csv")
        df2 = pd.read_csv(f"./train_combine_groupby/{l2}_traingroupby.csv")
        df1 = addcolumns(df1)
        df1 = combine_weather(df1)
        df1 = selecttime(df1)
        df2 = addcolumns(df2)
        df2 = combine_weather(df2)
        df2 = selecttime(df2)
        df1.dropna(inplace=True)
        df2.dropna(inplace=True)

        ###挑選兩個裝置(l1以及l2)  將兩個裝置同時段的資料取交集做merge，並拿l1的天氣資料當作X，l2的發電量當作Y來做訓練  會訓練出17*16個模型
        merge_df = pd.merge(
            df1,
            df2,
            left_on=['日期', '時間', '時段'],    # 左表連接的鍵
            right_on=['日期', '時間', '時段'], # 右表連接的鍵
            how='inner',
            suffixes=["_x","_y"]       #左表為訓練集的X  右表為訓練集的Y  
        )
        #以下為變數選擇  可自行調整  colnames後面要加上_x(挑選訓練集裝置(l1)的變數)
        X = merge_df.loc[:,["WindSpeed(m/s)_x","Pressure(hpa)_x", "Temperature(°C)_x","Humidity(%)_x","Sunlight(Lux)_x",
                            'SunshineDuration_x', 'UVIndex_x',
                            'AirTemperature_x', 'DailyExtremeHighAirTemperature_x',
                            'DailyExtremeLowAirTemperature_x', 'RelativeHumidity_x',
                            # 'VisibilityDescription_x',
                            "Power(mW)_x"]].reset_index(drop=True)
        Y = merge_df.loc[:,["Power(mW)_y"]].reset_index(drop=True)  #Y為訓練集裝置(l2)的發電量
        Y = Y.values.ravel()
        X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.01, random_state=42) #採用交叉驗證  拿全部資料來做訓練   自定義測試集僅需要少量資料供參考

        model = RandomForestRegressor(
            n_jobs=12,
        )# 可自行調整參數用 n_estimators= , max_depth=, min_samples_split=
        scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=5)
        MAE = -sum(scores)   #MAE以五折CV總和為基準
        model.fit(X_train, y_train)
       
        model_file_name = f'{l2}_{l1}.pkl'  #模型權重檔名命名模式：l2的模型的發電量用l1的變數預測的
        model_folder_name = "repeat_train_use_full_data_update_weather_data"         #將模型相關資訊儲存在這個資料夾下(在競賽結束後有重新測試過本程式碼因此在前面加上repeat_)
        model_file_path = os.path.join("./model_each_location",model_folder_name,model_file_name)
        os.makedirs(os.path.dirname(model_file_path), exist_ok=True)
        joblib.dump(model, model_file_path)  #將權重存檔
        print(model_file_name ," MAE:",MAE )
        print(f"模型已保存至: {model_file_path}")
        if l2 not in d:  #用字典紀錄每個模型的MAE以便預測時使用
            d[l2]={}
        d[l2][model_file_name] =MAE

02_01.pkl  MAE: 157.2229563793453
模型已保存至: ./model_each_location\repeat_train_use_full_data_update_weather_data\02_01.pkl
03_01.pkl  MAE: 188.67152996373355
模型已保存至: ./model_each_location\repeat_train_use_full_data_update_weather_data\03_01.pkl
04_01.pkl  MAE: 195.38599619311546
模型已保存至: ./model_each_location\repeat_train_use_full_data_update_weather_data\04_01.pkl
05_01.pkl  MAE: 259.9986455968603
模型已保存至: ./model_each_location\repeat_train_use_full_data_update_weather_data\05_01.pkl
06_01.pkl  MAE: 235.72654124042037
模型已保存至: ./model_each_location\repeat_train_use_full_data_update_weather_data\06_01.pkl
07_01.pkl  MAE: 169.59785442565135
模型已保存至: ./model_each_location\repeat_train_use_full_data_update_weather_data\07_01.pkl
08_01.pkl  MAE: 305.0717128129425
模型已保存至: ./model_each_location\repeat_train_use_full_data_update_weather_data\08_01.pkl
09_01.pkl  MAE: 272.1307859809594
模型已保存至: ./model_each_location\repeat_train_use_full_data_update_weather_data\09_01.pkl
10_01.pkl  MAE: 306.96755460

# 紀錄每個model的MAE並且存成json檔案

In [6]:
import json
json_output = json.dumps(d,indent=4)
output_path = f"./model_each_location/{model_folder_name}/d.json"
with open(output_path, "w", encoding="utf-8") as file:
    file.write(json_output)